# Import libraries

In [1]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

# Define constants

In [2]:
c = 299792.458 #is this in the correct units?
#H0 = 75 # might be able to find this myself?
f0 = 6.61* 10**-9 * 10**(-7) * (3.2408*10**(-25))**(-2)
megaparsec = 3.0856775814913673 * 10**10

# Some key functions

In [3]:
#order of variables: z, k, H0, omega_M, omega_DE, LPeakLambda, rc

def scale_factor(z):
    return 1/(1+z)

def curv_scale(H0, omega_M, omega_DE):
    R0 = c/(H0*np.sqrt(np.abs(omega_M+omega_DE-1)))
    return R0

def Friedmann_eq(z, k, H0, omega_M, omega_DE):
    a = scale_factor(z)
    R0 = curv_scale(H0, omega_M, omega_DE)
    H2 = (H0**2)*(omega_M/(a**3) + omega_DE - k*(c**2)/((R0**2)*(a**2)))
    return np.sqrt(H2)

def integrand(z, k, H0, omega_M, omega_DE):
    return c/Friedmann_eq(z, k, H0, omega_M, omega_DE)

def co_moving_dist(z_up, k, H0, omega_M, omega_DE):
    dist, err = scp.integrate.quad(integrand, 0, z_up, args = (k, H0, omega_M, omega_DE))
    return dist

def Sk(k, H0, omega_M, omega_DE, rc):
    R0 = curv_scale(H0, omega_M, omega_DE)
    if k == 0:
        sk = rc
    elif k > 0:
        sk = R0*np.sin(rc/R0)
    elif k < 0:
        sk = R0*np.sinh(rc/R0)
    return sk

def mag_model(z, k, H0, omega_M, omega_DE, LPeakLambda):
    rc = co_moving_dist(z, k, H0, omega_M, omega_DE)
    sk = Sk(k, H0, omega_M, omega_DE, rc)
    flux = LPeakLambda/(4*np.pi*(sk**2)*((1+z)**2))
    argument = flux/f0
    mag = -2.5*np.log10(argument)
    return mag

# Load the data

In [4]:
sn_z, sn_mag, sn_mag_err = np.loadtxt("sn_data.txt", usecols=(1,2,3), unpack = True)